In [1]:
! pip install transformers
! pip install huggingface_hub
! pip install torch torchvision torchaudio
# install AWS packages
! pip install boto3
! pip install sagemaker


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [2]:
# test pre-trained model
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TextClassificationPipeline

tokenizer = AutoTokenizer.from_pretrained("rwang5688/distilbert-base-uncased-finetuned-sst2")

model = AutoModelForSequenceClassification.from_pretrained("rwang5688/distilbert-base-uncased-finetuned-sst2")

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)
pipe("I love Amazon SageMaker Studio Lab!")

Downloading:   0%|          | 0.00/360 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/695k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

[[{'label': 'LABEL_0', 'score': 0.0031421473249793053},
  {'label': 'LABEL_1', 'score': 0.9968578815460205}]]

In [20]:
# set profile name as opposed to entering credentials
profile_name = 'default'
region_name = 'us-west-2'

In [21]:
# get and test sagemaker client
import boto3 
session = boto3.Session(profile_name=profile_name)
sm_client = session.client('sagemaker', region_name=region_name)
response = sm_client.list_endpoints()
print(response)

{'Endpoints': [{'EndpointName': 'sst2-text-classification-ep-2022-10-31-17-33-28', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:662235870471:endpoint/sst2-text-classification-ep-2022-10-31-17-33-28', 'CreationTime': datetime.datetime(2022, 10, 31, 11, 6, 40, 584000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2022, 10, 31, 11, 8, 46, 153000, tzinfo=tzlocal()), 'EndpointStatus': 'InService'}], 'ResponseMetadata': {'RequestId': 'ac2a2c93-9411-4ae6-a650-03b0f05600b3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'ac2a2c93-9411-4ae6-a650-03b0f05600b3', 'content-type': 'application/x-amz-json-1.1', 'content-length': '293', 'date': 'Fri, 11 Nov 2022 03:55:54 GMT'}, 'RetryAttempts': 0}}


In [7]:
# set model name and endpoint configuration name
import time
ml_model_name = "sst2-text-classification"
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
model_name = ml_model_name + '-model' + timestamp
endpoint_config_name = ml_model_name + '-epc' + timestamp
endpoint_name = ml_model_name + '-ep' + timestamp
print(model_name)
print(endpoint_config_name)
print(endpoint_name)

sst2-text-classification-model-2022-10-31-17-33-28
sst2-text-classification-epc-2022-10-31-17-33-28
sst2-text-classification-ep-2022-10-31-17-33-28


In [22]:
# set sagemaker execution role
import sagemaker
# create a sagemaker execution role via the AWS SageMaker console, then paste in the arn here
role = 'arn:aws:iam::662235870471:role/service-role/AmazonSageMaker-ExecutionRole-20221015T100906'

In [9]:
# see deep learning containers (DLC) available images here:
# https://github.com/aws/deep-learning-containers/blob/master/available_images.md 
model_image_url="763104351884.dkr.ecr."+region_name+".amazonaws.com/"+\
                "huggingface-pytorch-inference:1.9-transformers4.12-cpu-py38-ubuntu20.04"
print(model_image_url)

# set container config
container_config = {
    'Image': model_image_url,
    'Mode': 'SingleModel',
    'Environment': {
        'HF_MODEL_ID': 'rwang5688/distilbert-base-uncased-finetuned-sst2',
        'HF_TASK' : 'text-classification',
        'SAGEMAKER_CONTAINER_LOG_LEVEL' : '20',
        'SAGEMAKER_REGION' : region_name
    }
}
print(container_config)

# create model
# ... models console: https://console.aws.amazon.com/sagemaker/home?#/models
response = sm_client.create_model(
    ModelName=model_name,
    PrimaryContainer=container_config,
    ExecutionRoleArn=role, 
    EnableNetworkIsolation=False
)
print(response)

763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-inference:1.9-transformers4.12-cpu-py38-ubuntu20.04
{'Image': '763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-inference:1.9-transformers4.12-cpu-py38-ubuntu20.04', 'Mode': 'SingleModel', 'Environment': {'HF_MODEL_ID': 'rwang5688/distilbert-base-uncased-finetuned-sst2', 'HF_TASK': 'text-classification', 'SAGEMAKER_CONTAINER_LOG_LEVEL': '20', 'SAGEMAKER_REGION': 'us-west-2'}}
{'ModelArn': 'arn:aws:sagemaker:us-west-2:662235870471:model/sst2-text-classification-model-2022-10-31-17-33-28', 'ResponseMetadata': {'RequestId': '6d73564a-16b4-4030-a71e-75c887dde213', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '6d73564a-16b4-4030-a71e-75c887dde213', 'content-type': 'application/x-amz-json-1.1', 'content-length': '112', 'date': 'Mon, 31 Oct 2022 18:00:20 GMT'}, 'RetryAttempts': 0}}


In [10]:
# create endpoint config
# ... endpoint configs console: https://console.aws.amazon.com/sagemaker/home?#/endpointConfig
endpoint_config_response = sm_client.create_endpoint_config(
   EndpointConfigName=endpoint_config_name,
   ProductionVariants=[
        {
            "ModelName": model_name,
            "VariantName": "AllTraffic",
            "ServerlessConfig": {
                # Specify MemorySizeInMB and MaxConcurrency in the serverless config object
                "MemorySizeInMB": 3072,
                "MaxConcurrency": 10
            }
        }
    ]
)
print(endpoint_config_response)

print('Endpoint configuration name: {}'.format(endpoint_config_name))
print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:662235870471:endpoint-config/sst2-text-classification-epc-2022-10-31-17-33-28', 'ResponseMetadata': {'RequestId': '1a499d7f-cc39-4bcb-9a3d-44c19663066f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '1a499d7f-cc39-4bcb-9a3d-44c19663066f', 'content-type': 'application/x-amz-json-1.1', 'content-length': '129', 'date': 'Mon, 31 Oct 2022 18:06:09 GMT'}, 'RetryAttempts': 0}}
Endpoint configuration name: sst2-text-classification-epc-2022-10-31-17-33-28
Endpoint configuration arn:  arn:aws:sagemaker:us-west-2:662235870471:endpoint-config/sst2-text-classification-epc-2022-10-31-17-33-28


In [11]:
# create endpoint
# ... endpoints console: https://console.aws.amazon.com/sagemaker/home?#/endpoints
endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)
print(endpoint_response)

print('Endpoint name: {}'.format(endpoint_name))
print('Endpoint arn:  {}'.format(endpoint_response['EndpointArn']))

{'EndpointArn': 'arn:aws:sagemaker:us-west-2:662235870471:endpoint/sst2-text-classification-ep-2022-10-31-17-33-28', 'ResponseMetadata': {'RequestId': '5a177b65-0ff5-445b-a7c6-0e621533a15b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5a177b65-0ff5-445b-a7c6-0e621533a15b', 'content-type': 'application/x-amz-json-1.1', 'content-length': '115', 'date': 'Mon, 31 Oct 2022 18:06:39 GMT'}, 'RetryAttempts': 0}}
Endpoint name: sst2-text-classification-ep-2022-10-31-17-33-28
Endpoint arn:  arn:aws:sagemaker:us-west-2:662235870471:endpoint/sst2-text-classification-ep-2022-10-31-17-33-28


In [15]:
# WAIT FOR ENDPOINT TO BE "IN SERVICE" BEFORE PROCEEDING WITH THIS STEP

# invoke endpoint by endpoint name
import json
sm_runtime = session.client("sagemaker-runtime", region_name=region_name)

content_type = "application/json"

# specify "Inputs"
data = {
   "inputs": "I love Amazon SageMaker Studio Lab!"
}

response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=json.dumps(data)
)
print(response)
print(response["Body"].read().decode("utf-8"))

{'ResponseMetadata': {'RequestId': '67c2b543-58ac-4305-8fd5-e5d286acece6', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '67c2b543-58ac-4305-8fd5-e5d286acece6', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Mon, 31 Oct 2022 18:09:40 GMT', 'content-type': 'application/json', 'content-length': '48'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7ff121c9e6a0>}
[{"label":"LABEL_1","score":0.9968578815460205}]


In [ ]:
# clean up: uncomment the following lines
#sm_client.delete_endpoint(EndpointName=endpoint_name)
#sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
#sm_client.delete_model(ModelName=model_name)

In [ ]:
# build api
# !cd sm_api/
# !sam build
# !sam deploy --guided

# PyTorch Model Deployment

__Useful Resources:__

* [Real-time or Persistent Endpoint](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints.html)

* [Serverless Endpoint](https://docs.aws.amazon.com/sagemaker/latest/dg/serverless-endpoints-create.html)

* [Use PyTorch with Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/pytorch.html)

* [Deploy a Trained PyTorch Model Example](https://sagemaker-examples.readthedocs.io/en/latest/frameworks/pytorch/get_started_mnist_deploy.html)

* [Video walk-through of deploying a trained PyTorch Model](https://www.youtube.com/watch?v=ZrhUgjnDW8c)

* [Adding Custom Inference Scripts To SageMaker](https://aws.plainenglish.io/adding-custom-inference-scripts-to-amazon-sagemaker-2208c3332510)

__Create model file in S3:__

[Creating endpoint using PyTorch model trained somewhere else.](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html#bring-your-own-model) Typically, you save a PyTorch model as a file with extension `.pt` or `.pth`
* Write an inference script.
    * Save the inference script in the same folder where you saved your PyTorch model. Pass the filename of the inference script as the `entry_point` parameter when you create the `PyTorchModel` object.
* Create the directory structure for your model files.
```
| my_model 
|   |--model.pth 
|   
|   code 
|       |--inference.py 
|       |--requirements.txt
```
* Create the `PyTorchModel` object, and then call its `deploy()` method to deploy your model for inference. The `PyTorchModel` constructor packs files into a `tar.gz` file and uploads it to S3.
    
    ```python
    from sagemaker import get_execution_role
    role = get_execution_role()

    pytorch_model = PyTorchModel(model_data='s3://my-bucket/my-path/model.tar.gz', 
                                 role=role,
                                 entry_point='inference.py')

    predictor = pytorch_model.deploy(instance_type='ml.c4.xlarge', initial_instance_count=1)
    ```

In [6]:
import pandas as pd 
import torch
import sys
sys.path.append('../../deepfm/FuxiCTR-main/')
import fuxictr
fuxictr.__version__
from fuxictr.pytorch.models import DeepFM

In [18]:
model = torch.load("DeepFM_all_feature.model", map_location=torch.device('cpu'))

In [19]:
torch.jit.save(model, 'model_scripted.pt') # Save

AttributeError: 'collections.OrderedDict' object has no attribute 'save'

In [17]:
from torch.jit import save